In [1]:
import torch
from torchtext import data
from torchtext import datasets
import random, re

In [2]:
# clean comment_sentence, we can consider store the clean data instead
def clean_data(string):
    """
     Argument:
             string: the string to be cleaned
    """
    string = str(string)
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)     
    string = re.sub(r"\'s", " \'s", string) 
    string = re.sub(r"\'ve", " \'ve", string) 
    string = re.sub(r"n\'t", " n\'t", string) 
    string = re.sub(r"\'re", " \'re", string) 
    string = re.sub(r"\'d", " \'d", string) 
    string = re.sub(r"\'ll", " \'ll", string) 
    string = re.sub(r"\'m", " \'m", string)
    string = re.sub(r",", " , ", string) 
    string = re.sub(r"!", " ! ", string) 
    string = re.sub(r"\(", " \( ", string) 
    string = re.sub(r"\)", " \) ", string) 
    string = re.sub(r"\?", " \? ", string) 
    string = re.sub(r"\s{2,}", " ", string)    
    return string.strip().lower()

In [3]:
# Load a sample csv file test.csv, and use class TabularDataset to generate a torchtext.data.Dataset
file = "E:\Text_Classification\data\Yelp\yelp_review_full_csv"
TEXT = data.Field(tokenize='spacy',preprocessing=clean_data)
LABEL = data.LabelField(dtype=torch.int8)
tv_dataFields = [("label", LABEL), ("comment_text", TEXT)]
trn = data.TabularDataset.splits(path=file, train='test.csv',\
                                     format='csv',\
                                     skip_header=False, fields=tv_dataFields)

In [4]:
# Split the dataset into train and valid part, no need to do it manually
trn, val = trn[0].split(random_state=random.seed(1234))

In [5]:
# See the format of our dataset
val[0].__dict__

{'label': '3',
 'comment_text': "'they' , 'were' , 'solid' , ' ' , 'the' , 'po' , 'boy' , 'was' , 'good' , ' '"}

In [ ]:
# TODO: word2vec, load glove
TEXT.build_vocab(train_data, max_size=25000, vectors="glove.6B.100d")